In [1]:
import numpy as np
import random as rnd
import matplotlib.pyplot as plt
import math
import copy
import quadprog
from matplotlib import animation
plt.rc('animation', html='html5')

#### Initialization

In [110]:
dt = 0.1
agent1 = { "position": np.array([0.0,0.0]),
          "p_goal": np.array([20.0, 0.1]),
          "vel": np.array([1.1,0.0]),
          "radius": 0.5,
          "v_max": 1.1
    }
agent2 = { "position": np.array([20.0,0.0]),
          "p_goal": np.array([0.0,-0.1]),
          "vel": np.array([-1.1,0.0]),
          "radius": 0.5,
          "v_max": 1.1
    }

#### Animation

In [111]:
fig, ax = plt.subplots(figsize=(11,2))
ax.axes.set_xlim((-1, 21))
ax.axes.set_ylim((-2, 2))

circle1 = plt.Circle(agent1["position"], agent1["radius"], color='red')
circle2 = plt.Circle(agent2["position"], agent2["radius"], color='blue')
ax.add_artist(circle1)
ax.add_artist(circle2)

def animate(i):
    
    # Render
    circle1.center = agent1["position"]
    circle2.center = agent2["position"]
    
    if (rnd.random() < 0.5):

        # Update velocities
        v_pref1 = agent1["p_goal"]-agent1["position"]
        v_pref1 *= (agent1["v_max"]/length(v_pref1))
        u1,n1 = velObst(agent1, agent2, 1)
        G = 2.0*np.identity(2)
        a = 2.0*v_pref1
        C = np.reshape(n1, (1,2)).T
        b = np.array([np.sum(n1*(agent1["vel"] + 0.5*u1))])
        agent1["vel"] = quadprog.solve_qp(G, a, C, b)[0]

        v_pref2 = agent2["p_goal"]-agent2["position"]
        v_pref2 *= (agent2["v_max"]/length(v_pref2))
        u2,n2 = velObst(agent2, agent1, 1)
        a = 2.0*v_pref2
        C = np.reshape(n2, (1,2)).T
        b = np.array([np.sum(n2*(agent2["vel"] + 0.5*u2))])
        agent2["vel"] = quadprog.solve_qp(G, a, C, b)[0]
    else:
        G = 2.0*np.identity(2)
        v_pref2 = agent2["p_goal"]-agent2["position"]
        v_pref2 *= (agent2["v_max"]/length(v_pref2))
        u2,n2 = velObst(agent2, agent1, 1)
        a = 2.0*v_pref2
        C = np.reshape(n2, (1,2)).T
        b = np.array([np.sum(n2*(agent2["vel"] + 0.5*u2))])
        agent2["vel"] = quadprog.solve_qp(G, a, C, b)[0]
        
        # Update velocities
        v_pref1 = agent1["p_goal"]-agent1["position"]
        v_pref1 *= (agent1["v_max"]/length(v_pref1))
        u1,n1 = velObst(agent1, agent2, 1)
        a = 2.0*v_pref1
        C = np.reshape(n1, (1,2)).T
        b = np.array([np.sum(n1*(agent1["vel"] + 0.5*u1))])
        agent1["vel"] = quadprog.solve_qp(G, a, C, b)[0]

        
    # Update positions
    t = i*dt
    if (dist(circle1.center,agent1["p_goal"]) <= agent1["v_max"]*dt):
        agent1["position"] = agent1["p_goal"]
    else:
        agent1["position"] += dt*agent1["vel"]
    
    if (dist(circle2.center,agent2["p_goal"]) <= agent2["v_max"]*dt):
        agent2["position"] = agent2["p_goal"]
    else:
        agent2["position"] += dt*agent2["vel"]
    
    return circle1, circle2,

ani = animation.FuncAnimation(fig, animate, np.arange(0, 210), interval=(dt*500), blit=True)
ani.save('im.mp4')
ani

C:\Users\d1mas\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\d1mas\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in multiply
C:\Users\d1mas\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\d1mas\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in multiply
C:\Users\d1mas\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\d1mas\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in multiply
C:\Users\d1mas\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\d1mas\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered 

In [80]:
def velObst(a1, a2, tau):
    
    p1 = a1["position"]
    p2 = a2["position"]
    c0 = (p2-p1)/tau
    r0 = (a1["radius"]+a2["radius"])/tau
    
    c = length(c0)
    alpha = np.arcsin(r0/c)
    c_angle = np.arctan2(c0[1], c0[0])
    dist_ = np.sqrt(c*c-r0*r0)
    
    k1 = np.tan(c_angle+alpha)
    k2 = np.tan(c_angle-alpha)
    
    v_opt = a1["vel"]-a2["vel"]
    
    angle1 = c_angle+alpha
    angle1 = np.arctan2(np.sin(angle1), np.cos(angle1))
    angle2 = c_angle-alpha
    angle2 = np.arctan2(np.sin(angle2), np.cos(angle2))
    
    d1 = float("inf")
    x1 = 0
    y1 = 0
    proj_l = v_opt[0]*np.cos(angle1) + v_opt[1]*np.sin(angle1)
    if (proj_l > 0 and length(v_opt) > dist_):
        x1 = proj_l*np.cos(angle1)
        y1 = k1*x1
        d1 = dist(v_opt,(x1,y1))
    
    d2 = float("inf")
    x2 = 0
    y2 = 0
    proj_l = v_opt[0]*np.cos(angle2) + v_opt[1]*np.sin(angle2)
    if (proj_l > 0 and length(v_opt) > dist_):
        x2 = proj_l*np.cos(angle2)
        y2 = k2*x2
        d2 = dist(v_opt,(x2,y2))
    
    # Tangents to the circle
    k = (c0[1]-v_opt[1])/(c0[0]-v_opt[0])
    b = v_opt[1] - v_opt[0]*k
    a = 1+k*k
    eq_b = -2*c0[0]+2*k*b-2*k*c0[1]
    c = c0[0]*c0[0] + b*b + c0[1]*c0[1] - 2*b*c0[1] - r0*r0
    
    d_sqrt = np.sqrt(eq_b*eq_b - 4*a*c)
    x_sol1 = (-eq_b-d_sqrt)/(2*a)
    x_sol2 = (-eq_b+d_sqrt)/(2*a)
    y_sol1 = k*x_sol1+b
    y_sol2 = k*x_sol2+b
    
    d5 = float("inf")
    x5 = 0
    y5 = 0
    if (length((x_sol1,y_sol1)) < dist_):
        x5 = x_sol1
        y5 = y_sol1
        d5 = dist(v_opt,(x5,y5))
        
    d6 = float("inf")
    x6 = 0
    y6 = 0
    if (length((x_sol2,y_sol2)) < dist_):
        x6 = x_sol2
        y6 = y_sol2
        d6 = dist(v_opt,(x6,y6))
    
    # Find closest point
    u = None
    if (d1 <= d2 and d1 <= d5 and d1 <= d6):
        u = np.array([x1,y1]) - v_opt
    elif (d2 <= d1 and d2 <= d5 and d2 <= d6):
        u = np.array([x2,y2]) - v_opt
    elif (d5 <= d1 and d5 <= d2 and d5 <= d6):
        u = np.array([x5,y5]) - v_opt
    else:
        u = np.array([x6,y6]) - v_opt
    
    opt_in_obs = False
    opt_angle = np.arctan2(v_opt[1],v_opt[0])
    if (opt_angle > np.min((angle1, angle2)) and opt_angle < np.max((angle1, angle2))):
        if (length(v_opt) > dist_ or dist(v_opt, c0) < r0):
            opt_in_obs = True
    
    n = copy.deepcopy(u)
    if (not opt_in_obs):
        n = -n
    n /= length(n)
    
    return u, n

In [6]:
def dist(p1,p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
def length(p):
    return np.sqrt(p[0]*p[0] + p[1]*p[1])

#### Debugging

In [55]:
### TEST
tau = 1.0
agent1 = { "position": np.array([0.0,0.0]),
          "vel": np.array([4.5,1.5]),
          "radius": 0.5
    }
agent2 = { "position": np.array([2.0,2.0]),
          "vel": np.array([0.0,0.0]),
          "radius": 0.5
    }

In [56]:
velObst(agent1, agent2, tau)

c_angle: 0.785398163397
dist_: 2.64575131106
k1: 2.21525043702, k2: 0.451416229645
angle1: 1.1467652873, angle2: 0.424031039491
proj1_l: 3.2186269666
x1: 1.32426488753', y1: 2.93357837082
3.4843134833
proj2_l: 4.7186269666
x2: 4.30073511247', y2: 1.94142162918
0.484313483298
-1


(array([-0.19926489,  0.44142163]), array([ 0.41143783, -0.91143783]))

In [88]:
tau = 1
agent1 = { "position": np.array([0.0,0.0]),
          "p_goal": np.array([20.0, 0.0]),
          "vel": np.array([1.1,0.0]),
          "radius": 0.5,
          "v_max": 1.1
    }
agent2 = { "position": np.array([20.0,0.0]),
          "p_goal": np.array([0.0, 0.0]),
          "vel": np.array([-1.1,0.0]),
          "radius": 0.5,
          "v_max": 1.1
    }

In [91]:
velObst(agent1, agent2, tau)

(array([ 16.8,   0. ]), array([-1., -0.]))

In [92]:
u,n = velObst(agent1, agent2, tau)
agent1["vel"] + 0.5*u

array([ 9.5,  0. ])

In [100]:
G = 2.0*np.identity(2)
v_pref = agent1["p_goal"]-agent1["position"]
v_pref *= (agent1["v_max"]/length(v_pref))
print(v_pref)

[ 1.1  0. ]


In [101]:
a = 2.0*v_pref
C = np.reshape(n, (1,2)).T
b = np.array([np.sum(n*(agent1["vel"] + 0.5*u))])

In [102]:
quadprog.solve_qp(G, a, C, b)[0]

array([ 1.1,  0. ])

In [143]:
u1,n1 = velObst(agent1, agent2, dt)
u1

array([  1.87790001e+02,  -9.49441333e-02])

In [31]:
v_pref1 = agent1["p_goal"]-agent1["position"]
v_pref1 *= (agent1["v_max"]/length(v_pref1))
u1,n1 = velObst(agent1, agent2, dt)
    
G = np.reshape([[2.0,0.0],[0.0,2.0]], (2,2))
a = -2.0*v_pref1
C = np.reshape(n1, (1,2)).T
b = np.array([np.sum(n1*(agent1["vel"] + 0.5*u1))])
quadprog.solve_qp(G, a, C, b)[0]

array([ 0.,  0.])